In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Subset
from torchvision import datasets, models, transforms
from tempfile import TemporaryDirectory

In [ ]:
# Set number of classes
n_class = 5

# Set root path to the dataset
root_path = "/content/drive/MyDrive/4th year, 2nd sem/PD/Dataset/kneeKL224/"

# List all folders
folder_set = os.listdir(root_path)
image_path_list = []
label_list = []
folder_list = []

# For each folder, get the image path and labels
for folder in folder_set:
    for label in range(n_class):
        # Get all the images path inside the current folder
        image_list = os.listdir(f"{root_path}{folder}/{label}")
        # Add to the image path list
        image_path_list += [f"{root_path}{folder}/{label}/"+ path for path in image_list]
        # Add labels to the label list
        label_list += [label] * len(image_list)
        # Add folder to the folder list
        folder_list += [folder] * len(image_list)

# Convert to dataframe
dataset_dict = pd.DataFrame({"filepath" : image_path_list, "folder": folder_list, "label": label_list})

# Group by folder and label, and count filepaths
dataset_dict.groupby(["folder", "label"])["filepath"].count()

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataloaders = {}
dataset_sizes = {}
image_datasets = {x: datasets.ImageFolder(os.path.join(root_path, x), data_transforms[x]) for x in ['train', 'val', 'test']}
for sample in ['train', 'val', 'test']:
    idx = [i for i in range(len(image_datasets[sample])) if image_datasets[sample].imgs[i][1] not in [1,2,3]] # Exclude unused classes
    subset = Subset(image_datasets[sample], idx)
    dataloaders[sample] = torch.utils.data.DataLoader(subset, batch_size=4, shuffle=True, num_workers=4)
    dataset_sizes[sample] = len(idx)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models

# Load pre-trained DenseNet-201 model
model_densenet = models.densenet201(pretrained=True)

# Modify the final fully connected layer to match your task
num_ftrs = model_densenet.classifier.in_features
n_class = 5  # Assuming there are 5 classes
model_densenet.classifier = nn.Linear(num_ftrs, n_class)

# Set requires_grad to True for all parameters of the final layer
for param in model_densenet.classifier.parameters():
    param.requires_grad = True

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_densenet = model_densenet.to(device)

# Set up optimizer and loss function
optimizer_densenet = optim.SGD(model_densenet.parameters(), lr=0.001, momentum=0.9)
criterion_densenet = nn.CrossEntropyLoss()

# Set up learning rate scheduler if needed
exp_lr_scheduler_densenet = lr_scheduler.StepLR(optimizer_densenet, step_size=7, gamma=0.1)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:03<00:00, 26.2MB/s]


In [ ]:
import time
import os
import copy
import torch
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import datasets, transforms, models
import pandas as pd
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Initialize dictionaries to store metrics
    metrics = {'train': {'loss': [], 'acc': []}, 'val': {'loss': [], 'acc': []}}

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels)

                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                metrics[phase]['loss'].append(epoch_loss)
                metrics[phase]['acc'].append(epoch_acc)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:.4f}')

        model.load_state_dict(torch.load(best_model_params_path))

    # Plotting
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(metrics['train']['acc'], label='Train Accuracy')
    plt.plot(metrics['val']['acc'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(metrics['train']['loss'], label='Train Loss')
    plt.plot(metrics['val']['loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    return model


In [ ]:
model_densenet = train_model(model_densenet, criterion_densenet, optimizer_densenet,
                             exp_lr_scheduler_densenet, num_epochs=21)

Epoch 0/20
----------
train Loss: 0.2049 Acc: 0.9382
val Loss: 0.1022 Acc: 0.9549

Epoch 1/20
----------
train Loss: 0.1030 Acc: 0.9691
val Loss: 0.0491 Acc: 0.9915

Epoch 2/20
----------
train Loss: 0.0917 Acc: 0.9723
val Loss: 0.0155 Acc: 0.9915

Epoch 3/20
----------
train Loss: 0.0658 Acc: 0.9813
val Loss: 0.0313 Acc: 0.9915

Epoch 4/20
----------
train Loss: 0.0684 Acc: 0.9801
val Loss: 0.0185 Acc: 0.9972

Epoch 5/20
----------
train Loss: 0.0644 Acc: 0.9793
val Loss: 0.0129 Acc: 0.9944

Epoch 6/20
----------
train Loss: 0.0507 Acc: 0.9870
val Loss: 0.0079 Acc: 0.9972

Epoch 7/20
----------
train Loss: 0.0539 Acc: 0.9850
val Loss: 0.0051 Acc: 1.0000

Epoch 8/20
----------
train Loss: 0.0446 Acc: 0.9862
val Loss: 0.0070 Acc: 0.9972

Epoch 9/20
----------
train Loss: 0.0415 Acc: 0.9878
val Loss: 0.0054 Acc: 0.9972

Epoch 10/20
----------
